In [1]:
import json
import requests
import pandas as pd
from pandas.io.json import json_normalize
import matplotlib.pyplot as plt
import numpy as np
from sqlalchemy import create_engine
from sqlalchemy.ext.automap import automap_base
from sqlalchemy import create_engine, inspect, func
import datetime as dt
import sqlite3


In [2]:
url = "http://hotline.whalemuseum.org/api.json?&limit=1000&page="
page = 1

In [3]:
json = []

for page in range (1,20):
    query = url + str(page)
    response = requests.get(query)
    response_json = response.json()
    json.append(response_json)

In [4]:
i=0
for i in range (0,19): 
    if i == 0:
        df = pd.DataFrame.from_dict(json_normalize(json[i]), orient = 'columns')
    else: 
        df2 = pd.DataFrame.from_dict(json_normalize(json[i]), orient = 'columns')
        df = df.append(df2)

In [5]:
df = df.reindex()

df.head()


,id,species,quantity,description,url,latitude,longitude,location,sighted_at,created_at,updated_at,orca_type,orca_pod
0,5a049b7e686f743ec5040000,orca,5,"Spotted from the Edmonds-Kingston ferry, near ...",http://hotline.whalemuseum.org/sightings/5a049...,47.799530,-122.479967,,2017-11-09T18:11:00Z,2017-11-09T18:16:30Z,2017-11-14T22:20:45Z,unknown,NaN
1,59d039a0686f743ec5020000,harbor porpoise,20,From the Inn at Langley looking east there was...,http://hotline.whalemuseum.org/sightings/59d03...,48.047448,-122.404776,"Camano Island, WA, US",2017-10-01T00:38:00Z,2017-10-01T00:41:04Z,2017-10-03T22:01:43Z,unknown,NaN
2,59ab6d65686f747484140000,humpback,1,"heading north in the Tacoma narrows,, between ...",http://hotline.whalemuseum.org/sightings/59ab6...,47.307289,-122.561674,"Gig Harbor, WA, US",2017-09-03T00:45:00Z,2017-09-03T02:48:05Z,2017-09-06T18:17:03Z,unknown,NaN
3,597d058d686f7474840b0000,humpback,1,A large splash (breech?) first observed off Ke...,http://hotline.whalemuseum.org/sightings/597d0...,47.166960,-122.614546,"Steilacoom, WA, US",2017-07-29T21:55:00Z,2017-07-29T22:00:45Z,2017-08-03T19:52:25Z,unknown,NaN
4,596b516e686f747484090000,humpback,1,On train near picnic point (south of Mukilteo)...,http://hotline.whalemuseum.org/sightings/596b5...,47.884579,-122.342120,"Edmonds, WA, US",2017-07-16T02:39:00Z,2017-07-16T11:43:42Z,2017-08-03T19:51:46Z,unknown,NaN


In [6]:
df["month"]=pd.DatetimeIndex(df["sighted_at"]).month
df["year"]=pd.DatetimeIndex(df["sighted_at"]).year
df["hour"]=pd.DatetimeIndex(df["sighted_at"]).hour
df["year_month"]=pd.to_datetime(df["sighted_at"]).dt.to_period('M')

df.head()

C:\Users\cindy\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\arrays\datetimes.py:1269: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  UserWarning,


,id,species,quantity,description,url,latitude,longitude,location,sighted_at,created_at,updated_at,orca_type,orca_pod,month,year,hour,year_month
0,5a049b7e686f743ec5040000,orca,5,"Spotted from the Edmonds-Kingston ferry, near ...",http://hotline.whalemuseum.org/sightings/5a049...,47.799530,-122.479967,,2017-11-09T18:11:00Z,2017-11-09T18:16:30Z,2017-11-14T22:20:45Z,unknown,NaN,11,2017,18,2017-11
1,59d039a0686f743ec5020000,harbor porpoise,20,From the Inn at Langley looking east there was...,http://hotline.whalemuseum.org/sightings/59d03...,48.047448,-122.404776,"Camano Island, WA, US",2017-10-01T00:38:00Z,2017-10-01T00:41:04Z,2017-10-03T22:01:43Z,unknown,NaN,10,2017,0,2017-10
2,59ab6d65686f747484140000,humpback,1,"heading north in the Tacoma narrows,, between ...",http://hotline.whalemuseum.org/sightings/59ab6...,47.307289,-122.561674,"Gig Harbor, WA, US",2017-09-03T00:45:00Z,2017-09-03T02:48:05Z,2017-09-06T18:17:03Z,unknown,NaN,9,2017,0,2017-09
3,597d058d686f7474840b0000,humpback,1,A large splash (breech?) first observed off Ke...,http://hotline.whalemuseum.org/sightings/597d0...,47.166960,-122.614546,"Steilacoom, WA, US",2017-07-29T21:55:00Z,2017-07-29T22:00:45Z,2017-08-03T19:52:25Z,unknown,NaN,7,2017,21,2017-07
4,596b516e686f747484090000,humpback,1,On train near picnic point (south of Mukilteo)...,http://hotline.whalemuseum.org/sightings/596b5...,47.884579,-122.342120,"Edmonds, WA, US",2017-07-16T02:39:00Z,2017-07-16T11:43:42Z,2017-08-03T19:51:46Z,unknown,NaN,7,2017,2,2017-07


In [7]:

df = df.drop(columns =['url', 'quantity', 'created_at', 'updated_at' ])

In [8]:
df['hour_pst']=np.where(df['hour']<=8, df['hour'] + 16, df['hour']-8)


In [9]:
df['year_month']=df['year_month'].astype(str)

In [ ]:

df.head()

In [ ]:
df.columns


In [10]:
conn = sqlite3.connect("whale_watching.sqlite")
c = conn.cursor()

df.to_sql("Whales",conn,schema=None,if_exists='replace',index=False, index_label=None, chunksize=None, dtype=None)

#connect to the database


c.executescript('''
    PRAGMA foreign_keys=off;
    
    BEGIN TRANSACTION;
    
    ALTER TABLE "Whales" RENAME TO old_table2;

    /*create a new table with the same column names and types while
    defining a primary key for the desired column*/
    CREATE TABLE new_table (id TEXT PRIMARY KEY NOT NULL,
                            species TEXT, description TEXT, latitude DOUBLE , longitude DOUBLE, location TEXT,
       sighted_at TEXT, orca_type TEXt, orca_pod TEXT, month INT, year INT, hour INT,
       year_month TEXT, hour_pst INT);

    INSERT INTO new_table SELECT id, species, description, latitude, longitude, location,
       sighted_at,orca_type, orca_pod, month, year, hour,
       year_month, hour_pst FROM old_table2;

    DROP TABLE old_table2;
    COMMIT TRANSACTION;

    PRAGMA foreign_keys=on;''')

#close out the connection
c.close()
conn.close()

In [12]:
data = pd.read_sql("SELECT * FROM new_table", conn )
data.head(50)

ProgrammingError: Cannot operate on a closed database.

In [ ]:
# reflect an existing database into a new model
Base = automap_base()
# Base = declarative_base()

# reflect the tables
Base.prepare(engine, reflect = True)

In [ ]:
Base.classes.keys()

In [ ]:
inspector = inspect(engine)
inspector.get_table_names()
# Get a list of column names and types
columns_s = inspector.get_columns('Whales')
for c in columns_s:
    print(c['name'], c['type'])

In [ ]:
df.to_json(r'data.json', orient = 'records')

In [ ]:
df.count()

In [ ]:
df.head()

In [ ]:
locations = df.groupby(['location'])['id'].count()
print(locations)

In [ ]:
locations.plot(kind = "pie", autopct ="%1.1f%%")
plt.title("Whale Sightings by Location")
plt.show()

In [ ]:
#df["created_at"] = pd.to_datetime(df["created_at"])
df["month"]=pd.DatetimeIndex(df["sighted_at"]).year


df.head()

In [ ]:
month = df.groupby(["month"])["id"].count()
month.plot(kind="bar")

In [ ]:
species = df.groupby(["orca_pod"])["id"].count()
species.plot(kind="pie", autopct="%1.1f%%", startangle = 90)
plt.ylabel("")
plt.title("Whale Sightings by Pod")